In [1]:
# Next-layer emergence tests (triadic + dyadic), negative targets allowed
# ---------------------------------------------------------------
# What this cell does
# 1) Builds decimal-digit numbers from three sequences:
#    - FIB  : Fibonacci word -> decimal digits (0/1 as base-10 digits)
#    - RABBI: Rabbit (bitwise complement of FIB)
#    - FIBBI: binary expansion of (phi-1), then read those bits as base-10 digits
# 2) Reproduces the best-known near-hit α* = (3^6/10^3)*DF and its tiny residual to α.
# 3) “Second layer”: searches triadic + dyadic envelopes around α* to see
#    whether small 2-adic tweaks (±c/2^ℓ) systematically reduce errors or map α* to other constants.
# 4) Maps α* to other constants by fitting factors of the form:
#      factor ≈ 3^a * 2^b * 10^c * (1 ± d/2^ℓ),  with a,b,c small integers; d in {1,3}.
#    We test both +C and –C (negative targets are allowed).
# 5) Writes all tables to /content/out_next_layer/*.csv

from decimal import Decimal, getcontext
from math import sqrt, pi, e
import numpy as np
import pandas as pd
import os

# --- precision and helpers ---
getcontext().prec = 80

D = Decimal  # alias

def dec_pow_int(base, exp):
    # exact integer exponentiation for Decimal (supports negative exponents)
    if exp >= 0:
        return base ** exp
    else:
        return D(1) / (base ** (-exp))

def to_decimal_from_digits(digs):
    # Interprets a list/iterable of digits (0 or 1) as a base-10 decimal 0.d1 d2 d3...
    # Uses precise summation in Decimal.
    s = D(0)
    ten = D(10)
    p = D(1)
    for d in digs:
        p /= ten
        if d:
            s += p
    return s

# --- sequence generators ---
def fib_word_bits(n):
    # Fibonacci morphism: 0->01, 1->0, starting from '0'
    s = "0"
    while len(s) < n:
        s = s.replace("0", "a").replace("1", "b")  # placeholder to avoid overlap
        s = s.replace("a", "01").replace("b", "0")
    return [1 if ch == "1" else 0 for ch in s[:n]]

def rabbit_bits(n):
    # Rabbit morphism (bitwise complement of Fibonacci word by conjugacy):
    # 0->1, 1->10, starting from '1'
    s = "1"
    while len(s) < n:
        s = s.replace("0", "a").replace("1", "b")
        s = s.replace("a", "1").replace("b", "10")
    return [1 if ch == "1" else 0 for ch in s[:n]]

def frac_to_binary_bits(x, n):
    # binary expansion of fractional part of x in [0,1), n bits
    # classic multiply-by-2 method, using Decimal.
    bits = []
    r = x - int(x)  # keep fractional part
    for _ in range(n):
        r *= 2
        if r >= 1:
            bits.append(1)
            r -= 1
        else:
            bits.append(0)
    return bits

# --- constants and targets ---
phi = D(1) + (D(5).sqrt() - 1) / 2  # ensure Decimal; this equals (1+sqrt(5))/2
phi = (D(1) + D(5).sqrt()) / D(2)   # explicit golden ratio
alpha = D("0.0072973525693")         # fine-structure constant (reference value used in your runs)

targets = {
    "pi": D(str(pi)),
    "e": D(str(e)),
    "phi": phi,
    "phi-1": phi - 1,
    "1/pi": D(1)/D(str(pi)),
    "sqrt2": D(2).sqrt(),
    "sqrt3": D(3).sqrt(),
    "1/137": D(1)/D(137),
    "feigenbaum_delta": D("4.669201609102990671853203820466"),  # δ
    "feigenbaum_alpha": D("2.502907875095892822283902873218")   # α (scaling)
}

# --- build decimal-digit numbers (0/1 -> base-10 digits) ---
N = 2000  # number of digits; 1000–3000 is fine in Colab
F_bits = fib_word_bits(N)
R_bits = rabbit_bits(N)
# "FIBBI" = (phi-1) written in binary, then read those BITS as base-10 digits
FIBBI_bits = frac_to_binary_bits(phi - 1, N)

DF = to_decimal_from_digits(F_bits)
DR = to_decimal_from_digits(R_bits)
DFIBBI = to_decimal_from_digits(FIBBI_bits)

# check the complement identity with RABBI (should be 1/9 exactly, no carries)
sum_FR = DF + DR

# strongest near-hit around α
alpha_star = (dec_pow_int(D(3), 6) / (D(10) ** 3)) * DF  # (3^6/10^3)*DF
resid = alpha_star - alpha
step = DF / D(1000)    # one integer "click" in m (m/1000 scale); sub-step ratio in units of a click:
substep = resid / step

# independent analytic approximation: 101*(phi^(pi-2) - sqrt(3))
analytic_A = D(101) * (phi ** D(str(pi-2)) - D(3).sqrt())
analytic_err = analytic_A - alpha

print("=== Baseline checks ===")
print(f"Digits used N             : {N}")
print(f"DF + DR (should be 1/9)   : {sum_FR}   [1/9 = {D(1)/D(9)}]")
print(f"alpha* (3^6/1000*DF)      : {alpha_star}")
print(f"alpha_ref                 : {alpha}")
print(f"residual alpha*-alpha     : {resid}")
print(f'sub-step ratio resid/step : {substep}   (step = DF/1000 ≈ {step})')
print(f"analytic 101(phi^(pi-2)-sqrt3): {analytic_A}  (err {analytic_err})")

# --- triadic (3^k) scan around k=6 for the three sequences ---
def triadic_scan(DX, k_list):
    rows = []
    for k in k_list:
        est = dec_pow_int(D(3), k) / (D(10) ** 3) * DX
        rows.append({"k": k, "m=3^k": 3**k, "estimate": est, "error_to_alpha": est - alpha})
    return pd.DataFrame(rows)

k_list = list(range(0, 13))
scan_F   = triadic_scan(DF, k_list)
scan_R   = triadic_scan(DR, k_list)
scan_FBB = triadic_scan(DFIBBI, k_list)

# --- dyadic micro-envelopes around alpha_star: (1 ± c/2^ell), c in {1,3} ---
def dyadic_envelopes(center, cs=(1,3), max_ell=14):
    best = []
    for c in cs:
        for ell in range(2, max_ell+1):
            for sgn in (+1, -1):
                factor = D(1) + D(sgn*c) / dec_pow_int(D(2), ell)
                cand = center * factor
                best.append({
                    "c": c, "ell": ell, "sgn": sgn,
                    "factor": factor, "estimate": cand,
                    "abs_err": abs(cand - alpha),
                    "signed_err": cand - alpha
                })
    df = pd.DataFrame(best).sort_values("abs_err").reset_index(drop=True)
    return df

dyad = dyadic_envelopes(alpha_star)

print("\n=== Best dyadic micro-envelopes around alpha* ===")
print(dyad.head(10))

# --- map alpha* to other constants via (3^a * 2^b * 10^c) * (1 ± d/2^ell) ---
def fit_factor(from_value, to_value, a_range=range(-6,7), b_range=range(-12,13),
               c_range=range(-6,7), d_vals=(1,3), ell_range=range(2,13)):
    """
    Find a factor ~ 3^a * 2^b * 10^c * (1 ± d/2^ell) that carries from_value -> to_value
    by minimizing relative error |factor - target| / |target|,
    where target = to_value / from_value (try ± to_value).
    """
    results = []
    for sign_target in (+1, -1):  # allow negative targets
        target = D(sign_target) * (to_value / from_value)
        for a in a_range:
            tri = dec_pow_int(D(3), a)
            for b in b_range:
                dy  = dec_pow_int(D(2), b)
                for c in c_range:
                    dec = dec_pow_int(D(10), c)
                    base = tri * dy * dec
                    # raw (no micro term)
                    rel_err = abs(base - target) / abs(target)
                    results.append({
                        "sign_target": sign_target, "a": a, "b": b, "c": c,
                        "d": 0, "ell": None,
                        "factor": base, "rel_err": rel_err
                    })
                    # micro correction (1 ± d/2^ell)
                    for d in d_vals:
                        for ell in ell_range:
                            micro_plus  = base * (D(1) + D(d)/dec_pow_int(D(2), ell))
                            micro_minus = base * (D(1) - D(d)/dec_pow_int(D(2), ell))
                            relp = abs(micro_plus  - target) / abs(target)
                            relm = abs(micro_minus - target) / abs(target)
                            results.append({
                                "sign_target": sign_target, "a": a, "b": b, "c": c,
                                "d": d, "ell": ell, "factor": micro_plus, "rel_err": relp
                            })
                            results.append({
                                "sign_target": sign_target, "a": a, "b": b, "c": c,
                                "d": -d, "ell": ell, "factor": micro_minus, "rel_err": relm
                            })
    df = pd.DataFrame(results).sort_values("rel_err").reset_index(drop=True)
    return df

# run the mapping for a small catalog of constants
maps = []
for name, C in targets.items():
    fit = fit_factor(alpha_star, C)
    top = fit.head(12).copy()
    top["target"] = name
    top["approx_C_from_alpha*"] = (alpha_star * top["factor"]).astype(object)
    top["abs_err_C"] = (top["approx_C_from_alpha*"] - (top["sign_target"] * C)).abs().astype(object)
    maps.append(top)
maps_df = pd.concat(maps, ignore_index=True)

# --- make output directory and save ---
outdir = "/content/out_next_layer"
os.makedirs(outdir, exist_ok=True)

scan_F.to_csv(f"{outdir}/triadic_scan_F.csv", index=False)
scan_R.to_csv(f"{outdir}/triadic_scan_RABBI.csv", index=False)
scan_FBB.to_csv(f"{outdir}/triadic_scan_FIBBI.csv", index=False)
dyad.to_csv(f"{outdir}/dyadic_micro_envelopes.csv", index=False)
maps_df.to_csv(f"{outdir}/alpha_star_to_constants_map.csv", index=False)

print("\nSaved CSVs:")
for f in ["triadic_scan_F.csv", "triadic_scan_RABBI.csv", "triadic_scan_FIBBI.csv",
          "dyadic_micro_envelopes.csv", "alpha_star_to_constants_map.csv"]:
    print(" -", f"{outdir}/{f}")

# --- quick human-readable snapshots ---
print("\n=== Triadic scan around k=6 (FIB only, top 5 by |error|) ===")
print(scan_F.assign(abs_err=(scan_F["error_to_alpha"].abs())).sort_values("abs_err").head(5))

print("\n=== Top links: alpha* -> constants (by relative error), showing best 8 for each target ===")
def pretty_links(df, name, k=8):
    s = df[df["target"]==name].sort_values("rel_err").head(k)
    cols = ["sign_target","a","b","c","d","ell","rel_err","abs_err_C"]
    print(f"\nTarget: {name}")
    print(s[cols].to_string(index=False))
for name in targets.keys():
    pretty_links(maps_df, name)


=== Baseline checks ===
Digits used N             : 2000
DF + DR (should be 1/9)   : 0.11111111111111111111111111111111111111111111111111111111111111111111111111111111   [1/9 = 0.11111111111111111111111111111111111111111111111111111111111111111111111111111111]
alpha* (3^6/1000*DF)      : 0.0072973629729736297362972973629729736297362972973629736297297362972973629736297297
alpha_ref                 : 0.0072973525693
residual alpha*-alpha     : 1.04036736297362972973629729736297362972973629736297297362972973629736297297E-8
sub-step ratio resid/step : 0.0010393176417517867944022678319131602492852454675439696646284545414141312986970770   (step = DF/1000 ≈ 0.000010010100100101001010010010100100101001010010010100101001001010010010100101001001)
analytic 101(phi^(pi-2)-sqrt3): 0.0073020238070871255551831404155954285586191477537074527680691375871265444965548  (err 0.0000046712377871255551831404155954285586191477537074527680691375871265444965548)

=== Best dyadic micro-envelopes around alpha* ===